Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

C:\Users\o.ivashkin\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.112322
Minibatch accuracy: 12.5%
Validation accuracy: 10.9%
Minibatch loss at step 50: 2.200843
Minibatch accuracy: 31.2%
Validation accuracy: 17.7%
Minibatch loss at step 100: 1.549535
Minibatch accuracy: 37.5%
Validation accuracy: 47.1%
Minibatch loss at step 150: 1.180979
Minibatch accuracy: 75.0%
Validation accuracy: 72.0%
Minibatch loss at step 200: 0.255959
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 250: 0.996414
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 300: 0.887719
Minibatch accuracy: 68.8%
Validation accuracy: 76.5%
Minibatch loss at step 350: 1.176035
Minibatch accuracy: 68.8%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.964417
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 450: 0.290928
Minibatch accuracy: 93.8%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.264620
Minibatch accuracy: 87.5%
Validation accuracy: 79.7%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
 
  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))

  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    hidden = tf.nn.max_pool(data, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    hidden = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [19]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.461619
Minibatch accuracy: 18.8%
Validation accuracy: 9.9%
Minibatch loss at step 50: 2.061605
Minibatch accuracy: 43.8%
Validation accuracy: 41.1%
Minibatch loss at step 100: 1.752623
Minibatch accuracy: 37.5%
Validation accuracy: 62.4%
Minibatch loss at step 150: 1.341750
Minibatch accuracy: 68.8%
Validation accuracy: 63.1%
Minibatch loss at step 200: 0.927977
Minibatch accuracy: 75.0%
Validation accuracy: 67.8%
Minibatch loss at step 250: 1.286076
Minibatch accuracy: 81.2%
Validation accuracy: 71.1%
Minibatch loss at step 300: 1.054884
Minibatch accuracy: 56.2%
Validation accuracy: 72.2%
Minibatch loss at step 350: 1.075979
Minibatch accuracy: 56.2%
Validation accuracy: 71.3%
Minibatch loss at step 400: 1.332936
Minibatch accuracy: 68.8%
Validation accuracy: 70.9%
Minibatch loss at step 450: 0.720802
Minibatch accuracy: 81.2%
Validation accuracy: 74.0%
Minibatch loss at step 500: 0.535882
Minibatch accuracy: 75.0%
Validation accuracy: 74.0%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [23]:
batch_size = 128

patch_conv = 5
patch_pool = 2
depth1 = 6
depth2 = 6
depth3 = 16
depth4 = 16
depth5 = 120

num_hidden6 = 84

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    
    w1 = tf.Variable(tf.truncated_normal([patch_conv, patch_conv, num_channels, depth1], stddev=0.1))
    b1 = tf.Variable(tf.zeros([depth1]))
    
    w3 = tf.Variable(tf.truncated_normal([patch_conv, patch_conv, depth1, depth3], stddev=0.1))
    b3 = tf.Variable(tf.zeros([depth3]))
    
    w5 = tf.Variable(tf.truncated_normal([patch_conv, patch_conv, depth3, depth5], stddev=0.1))
    b5 = tf.Variable(tf.zeros([depth5]))
    
    w6 = tf.Variable(tf.truncated_normal([1 * 1 * depth5, num_hidden6], stddev=0.1))
    b6 = tf.Variable(tf.constant(1.0, shape=[num_hidden6]))
    
    w7 = tf.Variable(tf.truncated_normal([num_hidden6, num_labels], stddev=0.1))
    b7 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    
    # Model.
    def model(data):
        c1 = tf.nn.conv2d(data, w1, [1, 1, 1, 1], padding='SAME')
        c1 = tf.tanh(c1 + b1)
        
        s2 = tf.nn.avg_pool(c1, ksize=[1, patch_pool, patch_pool, 1], strides=[1, 2, 2, 1], padding='SAME')
        c1 = tf.tanh(c1 + b1)
        
        c3 = tf.nn.conv2d(s2, w3, [1, 1, 1, 1], padding='VALID')
        c3 = tf.tanh(c3 + b3)
        
        s4 = tf.nn.avg_pool(c3, ksize=[1, patch_pool, patch_pool, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        c5 = tf.nn.conv2d(s4, w5, [1, 1, 1, 1], padding='VALID')
        c5 = tf.tanh(c5 + b5)
        shape5 = c5.get_shape().as_list()
        reshape5 = tf.reshape(c5, [shape5[0], shape5[1] * shape5[2] * shape5[3]])
        
        f6 = tf.tanh(tf.matmul(reshape5, w6) + b6)
        
        return tf.matmul(f6, w7) + b7
        
        
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [24]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.338388
Minibatch accuracy: 7.0%
Validation accuracy: 10.1%
Minibatch loss at step 50: 1.049398
Minibatch accuracy: 73.4%
Validation accuracy: 75.7%
Minibatch loss at step 100: 0.889035
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 150: 0.668634
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 200: 0.667786
Minibatch accuracy: 85.2%
Validation accuracy: 82.3%
Minibatch loss at step 250: 0.808838
Minibatch accuracy: 77.3%
Validation accuracy: 82.5%
Minibatch loss at step 300: 0.574016
Minibatch accuracy: 85.2%
Validation accuracy: 82.9%
Minibatch loss at step 350: 0.616497
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 400: 0.549190
Minibatch accuracy: 85.9%
Validation accuracy: 83.9%
Minibatch loss at step 450: 0.503084
Minibatch accuracy: 85.2%
Validation accuracy: 83.9%
Minibatch loss at step 500: 0.725979
Minibatch accuracy: 77.3%
Validation accuracy: 84.4%
Mi